In [35]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import time 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from itertools import cycle
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import train_test_split, KFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

from xgboost import XGBRegressor
from sklearn.metrics import r2_score


In [22]:
train_all = pd.read_csv('train100900.csv')
test_data = pd.read_csv('test_full.csv')
val_all = pd.read_csv('val_full.csv')

In [23]:
#train_all = train_all.drop(train_all.columns[[0,1]],axis =1)
train_all= train_all.drop(['region','city','counts_r','counts_c','Unnamed: 0','index'],axis =1)
train_all.head(5)

,OS_android,OS_ios,OS_linux,OS_mac,OS_other,OS_windows,adexchange,advertiser,bidprice,browser_chrome,...,usertag_13874,usertag_14273,usertag_15398,usertag_16593,usertag_16617,usertag_16661,usertag_16706,usertag_16751,usertag_16753,weekday
0,0,0,0,0,0,1,2,2259,277,1,...,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,3,2821,294,0,...,0,1,0,0,0,0,0,0,0,2
2,1,0,0,0,0,0,0,2997,277,0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,0,0,0,1,3,3427,241,0,...,0,0,0,0,0,0,0,0,0,2
4,0,0,0,0,0,1,3,1458,300,0,...,0,0,0,0,0,0,0,0,0,2


In [24]:
#test_data = test_data.drop(test_data.columns[[0]],axis =1)
test_data= test_data.drop(['region','city','counts_r','counts_c','Unnamed: 0'],axis =1)
test_data.head(5)

,weekday,hour,adexchange,slotwidth,slotheight,slotprice,advertiser,usertag_10024,usertag_10077,usertag_10075,...,browser_firefox,browser_ie,browser_maxthon,browser_opera,browser_other,browser_safari,browser_sogou,browser_theworld,freq_c,freq_r
0,0,12,3,300,250,10,3427,1,1,1,...,0,0,0,0,0,0,0,0,0.004638,0.065971
1,3,14,0,320,50,5,2997,0,0,0,...,0,0,0,0,0,1,0,0,0.041582,0.041582
2,5,19,1,336,280,0,1458,1,0,0,...,0,1,0,0,0,0,0,0,0.001335,0.012635
3,0,21,4,960,90,30,2821,0,0,1,...,0,1,0,0,0,0,0,0,0.000910,0.023984
4,2,20,3,300,250,50,3386,0,0,0,...,0,1,0,0,0,0,0,0,0.000722,0.012635


In [25]:
#val_all = val_all.drop(val_all.columns[[0,1]],axis =1)
val_all= val_all.drop(['region','city','counts_r','counts_c','Unnamed: 0','index'],axis =1)
val_all.head(5)

,weekday,hour,adexchange,slotwidth,slotheight,slotprice,bidprice,payprice,advertiser,click,...,browser_firefox,browser_ie,browser_maxthon,browser_opera,browser_other,browser_safari,browser_sogou,browser_theworld,freq_c,freq_r
0,4,20,1,160,600,0,300,23,1458,0,...,0,1,0,0,0,0,0,0,0.026276,0.026276
1,1,21,1,950,90,0,238,75,3476,0,...,0,0,0,0,0,0,0,0,0.026276,0.026276
2,4,8,2,300,250,5,238,65,3358,0,...,0,1,0,0,0,0,0,0,0.014994,0.014994
3,5,15,2,336,280,5,238,6,3358,0,...,0,0,0,0,0,0,0,0,0.002172,0.027197
4,1,18,2,200,200,5,249,5,3476,0,...,0,0,0,0,0,0,0,0,0.005459,0.019347


In [26]:
train_all_1 = train_all.reindex_axis(sorted(train_all.columns), axis=1)
#list(train_all_1)
train_all_1.shape

/Users/jennyyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """Entry point for launching an IPython kernel.


(709200, 103)

In [27]:
test_data_1 = test_data.reindex_axis(sorted(test_data.columns), axis=1)
#list(test_data_1)
test_data_1.shape

/Users/jennyyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """Entry point for launching an IPython kernel.


(303375, 100)

In [28]:
val_all_1 = val_all.reindex_axis(sorted(val_all.columns), axis=1)
#list(val_all_1)
val_all_1.shape

/Users/jennyyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: '.reindex_axis' is deprecated and will be removed in a future version. Use '.reindex' instead.
  """Entry point for launching an IPython kernel.


(303925, 103)

In [29]:
#100 features
train_y_ctr = train_all_1['click']
train_y_wp = train_all_1['payprice']
train_x = train_all_1.drop(['click','payprice','bidprice'],axis =1)
test_y_ctr = val_all_1['click']
test_y_wp = val_all_1['payprice']
test_x = val_all_1.drop(['click','payprice','bidprice'],axis =1)

print(train_y_ctr.shape)
print(train_y_wp.shape)
print(train_x.shape)
print(test_y_ctr.shape)
print(test_y_wp.shape)
print(test_x.shape)

(709200,)
(709200,)
(709200, 100)
(303925,)
(303925,)
(303925, 100)


# TUNE THE MODEL

In [30]:
X_trian_A = train_x
y_train_A = train_y_ctr

In [ ]:
depth = [3,4]
learning = [0.001,0.05,0.1,0.15,0.2,0.3]
estimators = [100,150,200,300]
#----------------------------------

model = XGBClassifier()
param_grid = dict(max_depth = depth,learning_rate=learning, n_estimators=estimators)
kfold = StratifiedKFold(n_splits=4, shuffle=True)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold,verbose=True)
grid_result = grid_search.fit(X_trian_A, y_train_A)

# Part 1 results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#{'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 150}

In [ ]:
gamma = [0,0.1,0.2,0.3,0.4,0.5]
min_child_weight =[1,3,5,7,9]
#----------------------------------

model = XGBClassifier(max_depth=3,n_estimators=150,learning_rate=0.1) #tune 1 result
param_grid = dict(gamma = gamma,min_child_weight=min_child_weight)
kfold = StratifiedKFold(n_splits=4, shuffle=True)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold,verbose=True)
grid_result = grid_search.fit(X_trian_A, y_train_A)

# Part 2 results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
#{'gamma': 0.1, 'min_child_weight': 1}

In [27]:
subsample = [0.85,0.9,0.95]
colsample_bytree=[0.55,0.6,0.65]
#----------------------------------

model = XGBClassifier(max_depth=3,
                      n_estimators=150,
                      learning_rate=0.1,
                      gamma = 0.1,
                      min_child_weight =1,
                      random_state=0)       #tune 2 result

param_grid = dict(subsample = subsample,colsample_bytree=colsample_bytree)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold,verbose=True)
grid_result = grid_search.fit(X_trian_A, y_train_A)

# part 3 results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 9 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed:  2.2min finished


Best: -0.206398 using {'colsample_bytree': 0.55, 'subsample': 0.9}
-0.207845 (0.023424) with: {'colsample_bytree': 0.55, 'subsample': 0.85}
-0.206398 (0.024225) with: {'colsample_bytree': 0.55, 'subsample': 0.9}
-0.209670 (0.023479) with: {'colsample_bytree': 0.55, 'subsample': 0.95}
-0.208707 (0.022544) with: {'colsample_bytree': 0.6, 'subsample': 0.85}
-0.206800 (0.022451) with: {'colsample_bytree': 0.6, 'subsample': 0.9}
-0.210511 (0.023429) with: {'colsample_bytree': 0.6, 'subsample': 0.95}
-0.208887 (0.024954) with: {'colsample_bytree': 0.65, 'subsample': 0.85}
-0.207307 (0.024327) with: {'colsample_bytree': 0.65, 'subsample': 0.9}
-0.208788 (0.025144) with: {'colsample_bytree': 0.65, 'subsample': 0.95}


In [11]:
#{'colsample_bytree': 0.6, 'subsample': 0.9}

#{'colsample_bytree': 0.55, 'subsample': 0.9}

[0.0, 0.1, 0.2, 0.3, 0.4]

In [29]:
reg_alpha = [0.03,0.04,0.05,0.06,0.07]


model = XGBClassifier(max_depth=3,
                      n_estimators=150,
                      learning_rate=0.1,
                      gamma = 0.1,
                      min_child_weight =1,
                      random_state=0,
                      colsample_bytree=0.55,
                      subsample=0.9
                        )

param_grid = dict(reg_alpha=reg_alpha)

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold,verbose=True)
grid_result = grid_search.fit(X_trian_A, y_train_A)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 5 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  38 out of  50 | elapsed:  1.1min remaining:   20.3s
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:  1.1min finished


Best: -0.204953 using {'reg_alpha': 0.05}
-0.205661 (0.022168) with: {'reg_alpha': 0.03}
-0.205446 (0.022219) with: {'reg_alpha': 0.04}
-0.204953 (0.022710) with: {'reg_alpha': 0.05}
-0.206368 (0.023172) with: {'reg_alpha': 0.06}
-0.207059 (0.024609) with: {'reg_alpha': 0.07}


In [ ]:
#Best: -0.204953 using {'reg_alpha': 0.05}